In [149]:
from nltk.tokenize import sent_tokenize, word_tokenize
from sematch.semantic.similarity import YagoTypeSimilarity
from sematch.semantic.graph import DBpediaDataTransform, Taxonomy
from sematch.semantic.similarity import ConceptSimilarity
import numpy as np
import scipy as sc

In [62]:
stopwords = nltk.corpus.stopwords.words('english')
swset = set(stopwords)
sim = YagoTypeSimilarity()
concept = ConceptSimilarity(Taxonomy(DBpediaDataTransform()),'models/dbpedia_type_ic.txt')

In [71]:
from sematch.semantic.similarity import WordNetSimilarity
wns = WordNetSimilarity()




In [84]:
sim.word2yago('drink')

[u'http://dbpedia.org/class/yago/Drink107885223',
 u'http://dbpedia.org/class/yago/Drink100748515',
 u'http://dbpedia.org/class/yago/Beverage107881800',
 u'http://dbpedia.org/class/yago/Drink109270508',
 u'http://dbpedia.org/class/yago/Swallow100839778']

0.14285714285714285

In [2]:
sim.yago_similarity('http://dbpedia.org/class/yago/Car','http://dbpedia.org/class/yago/Automobile', 'wpath') #0.544

0.0

In [3]:
from sematch.application import Matcher
matcher = Matcher()

In [75]:
matcher.match_concepts('Car')

[]

In [47]:
def load_stss131():
    import csv
    f = open('datasets/stss-131/stss.csv', 'r')
    reader = csv.reader(f, delimiter=';')
    rows = []
    for row in reader: rows.append(row)
    return rows

def tokenize(sentence,swset):
    return [w for w in word_tokenize(sentence) if w not in swset]

In [13]:
stss131 = load_stss131()

In [135]:
a = stss131[0][1]
b = stss131[0][2]

def yagonize(sentence):
    yagos = []
    for word in tokenize(sentence, swset):
        yago_a_options = sim.word2yago(word)
        yago_a = yago_a_options[0] if len(yago_a_options) > 0 else False
        if yago_a:
            yagos.append(yago_a)
    return yagos

def compare_yagos(a,b, method='wpath'):
    yagos = dict()
    
    for y_a in a:    
        results_y_a = list()
        
        for y_b in b:
            results_y_a.append(sim.yago_similarity(y_a,y_b, method))
            
        yagos[y_a] = results_y_a
        
    return yagos

def yago_mean(r):
    m = list()
    for k,v in r.iteritems():
        m.append(max(v))
    return np.mean(m)
        
results_a_to_b = compare_yagos(yagonize(a),yagonize(b))

In [128]:
def yago_mean(r):
    m = list()
    for k,v in r.iteritems():
        m.append(max(v))
    return np.mean(m)

http://dbpedia.org/class/yago/Drink107885223
http://dbpedia.org/class/yago/Go115292069
http://dbpedia.org/class/yago/Like105845888
http://dbpedia.org/class/yago/Tonight115263045


0.576759025027096

In [129]:
results_b_to_a = compare_yagos(yagonize(b), yagonize(a))

In [130]:
m = list()
for k,v in results_b_to_a.iteritems():
    print(k)
    m.append(max(v))
np.mean(m)

http://dbpedia.org/class/yago/Might105030680
http://dbpedia.org/class/yago/Tonight115263045
http://dbpedia.org/class/yago/Go115292069
http://dbpedia.org/class/yago/Iodine114641397
http://dbpedia.org/class/yago/Somewhere108489765
http://dbpedia.org/class/yago/Know105805012


0.4351463296960698

In [136]:
def yago_sim_score(s1,s2, m='wpath'):
    y_l = yagonize(s1)
    y_r = yagonize(s2)
    
    l2r = compare_yagos(y_l, y_r, m)
    r2l = compare_yagos(y_r, y_l, m)
    
    l2r_score = yago_mean(l2r)
    r2l_score = yago_mean(r2l)
    
    return np.mean([l2r_score, r2l_score])
    

0.5059526773615829

In [146]:
sentences_with_score = list()
method = 'wpath'
for row in stss131:
    s1 = row[1]
    s2 = row[2]
    try:
        sentences_with_score.append(
            [s1,s2, yago_sim_score(s1,s2,method)]
        )
    except:
        sentences_with_score.append(
            [s1,s2, None]
        )

In [152]:
yago_s = [s for _,_,s in sentences_with_score]

In [153]:
stss_s = [r[3] for r in stss131]

In [156]:
cleaned_s = [ t for t in zip(yago_s, stss_s) if t[0] != None]

In [160]:
yago_final, stss_final = list(zip(*cleaned_s))

In [161]:
np.corrcoef(yago_final, stss_final)

TypeError: cannot perform reduce with flexible type